# Project: Wrangling and Analyze Data

In [11]:
!pip install --upgrade tweepy==4.6.0

Requirement already up-to-date: tweepy==4.6.0 in /opt/conda/lib/python3.6/site-packages (4.6.0)


Importing Libraries

In [1]:
import tweepy

In [2]:
tweepy.__version__

'4.6.0'

In [3]:
import requests #download data
import numpy as np #array functions
import pandas as pd #data handling
import tweepy #twitter api
import json #handle json data
import matplotlib.pyplot as plt #data visualization
import seaborn as sns #data visualization
import re #text processing
import time

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly downloading the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv) and displaying the top 5 rows

**1. Loading the Dataset**

In [4]:
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
twitter_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


**2. Image Predictions**

First defining the url file and then using the request.get function, and write the content of the response to a new file. 

In [5]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

#get response
response = requests.get(url)

#write return to an image
with open("image_predictions.tsv", mode = "wb") as file:
    file.write(response.content)

In [6]:
twitter_predict = pd.read_csv("image_predictions.tsv", sep='\t')
twitter_predict.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


**3. Gathering additional data with the Twitter API**

In [7]:
access_token = "1556583358372642816-EBDvb4QsFo4xeYFdnT8zO0neKhTuq9"
access_secret = "lNUpIXE9ENNUjCbVbFDIqQ5GS9DavCKgeYOBtuLL5928W"
consumer_key = "aPBkWyvtzGV41yl17OLMlXeDa"
consumer_secret = "vk9mGd5ls7CUH2Mq7o5auhLoxuCwODmmwxcHm040bZBPjpBWo3"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAAfzfgEAAAAA9CDd58pGvknfdOlSG3Dn%2FGAawtI%3DEUEcRid0Z48ojEMLKBzlg6AfXsRC7sBHNYqjLSWtZfR3xiOS54"
client_id="SklVakRsZkxKNi1QOFAwdUQ1WUo6MTpjaQ"
client_secret="p3GGBNbylckZbFqtv5yazdCv4cgh-tmORMedgujFZ8sSx3mFBr"

In [8]:
client = tweepy.Client(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_secret
)

In [9]:
client.get_me()

Response(data=<User id=1556583358372642816 name=Jitka_Richterova username=JitkaRichter>, includes={}, errors=[], meta={})

Getting all IDs in Twitter Dataset

In [11]:
twitter_ids = list(twitter_archive.tweet_id.unique())

# Tweet IDs for which to gather additional data via Twitter's API
twitter_ids = twitter_archive.tweet_id.values
len(twitter_ids)

2356

In [25]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
api

In [28]:
auth = tweepy.OAuthHandler(client_id, client_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
api

additional source:  https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id

In [26]:
 # Creating a for loop to add the tweets to a new line of the datafile 
with open('tweet_json.txt', 'a', encoding='utf8') as f:
    for id in twitter_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, f)
            f.write('\n')
        except:
            continue

In [27]:
# Creating a for loop to add the tweets to a new line of the datafile 
with open('tweet_json.txt', 'a', encoding='utf8') as f:
    for id in twitter_ids:
        try:
            tweet = api.get_status(twitter_archive.tweet_id, tweet_mode='extended')
            json.dump(tweet._json, f)
            f.write('\n')
        except Exception as e:
            print('Something bad happened')
            print(e)

Something bad happened
403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve
Something bad happened
403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve
Something bad happened
403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://de

KeyboardInterrupt: 

In [12]:
twitter_list = []

# Read the .txt file line by line into a list of dictionaries
for line in open('tweet_json.txt', 'r'):
    twitter_data = json.loads(line)
    twitter_list.append({'tweet_id': twitter_data['id_str'],
                        'retweet_count': twitter_data['retweet_count'],
                        'favorite_count': twitter_data['favorite_count'],
                        'followers_count': twitter_data['user']['followers_count']})
    

In [31]:
 # Convert the list of dictionaries to a pandas DataFrame
twitter_df = pd.DataFrame(twitter_list, columns = ['tweet_id', 'retweet_count', 'favorite_count', 'followers_count'])

In [32]:
twitter_df.head(5)

,tweet_id,retweet_count,favorite_count,followers_count


## attempt three

In [12]:
import tweepy 

# defining bearer token received through developed access for twitter to get the twitter data via Client
bearer_token='AAAAAAAAAAAAAAAAAAAAAAfzfgEAAAAAASzZc%2FA2shpjxfzSMAcEk0aMfbY%3DAntBgueRHyGMxO6ag3FrHbY5Uqx48BCBG84ZQQSmVnYHqPE7sH'

# Initialising Client

client = tweepy.Client(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_secret,
    bearer_token=bearer_token)

tweet_ids=twitter_archive.tweet_id.values

#trying out code for one example tweet ID
example_tweet=tweet_ids[0]

print(f'Quering API for tweets: {example_tweet}\n') 

# retweet and favourites are in public_metrics
tweet_fields=['created_at', 'id', 'public_metrics','source', 'text']

# get information on individual tweet - example_tweet
temp=client.get_tweet(id=example_tweet, tweet_fields=tweet_fields)

# store information in .data
print(f'{temp.data}\n') 

#if temp.data.get('public_metrics'):
 #   print(f"{temp.data['public_metrics']=}") 


Quering API for tweets: 892420643555336193



Unauthorized: 401 Unauthorized

In [14]:
# running the code for whole twitter ids
# Initialising Client
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

tweet_ids=twitter_archive.tweet_id.values

print(f'Quering API for tweets: {example_tweet}\n') 

# retweet and favourites are in public_metrics
tweet_fields=['created_at', 'id', 'public_metrics','source', 'text']

# get information on individual tweet - example_tweet
temp=client.get_tweet(id=tweet_ids, tweet_fields=tweet_fields)

# store information in .data
print(f'{temp.data}\n') 

#if temp.data.get('public_metrics'):
 #   print(f"{temp.data['public_metrics']=}") 


Quering API for tweets: 892420643555336193



BadRequest: 400 Bad Request

In [15]:
# store data
tweet_data={}
if temp.data.get('public_metrics'):
    for key, value in temp.data['public_metrics'].items():
        #store the results
        tweet_data[key]=value
        
tweet_data

NameError: name 'temp' is not defined

In [16]:
import tweepy
import time
import json

bearer_token=['AAAAAAAAAAAAAAAAAAAAAAfzfgEAAAAA9CDd58pGvknfdOlSG3Dn%2FGAawtI%3DEUEcRid0Z48ojEMLKBzlg6AfXsRC7sBHNYqjLSWtZfR3xiOS54']
# initialise .Client()
client=tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

info=['public_metrics']

tweet_info=[]

# Loop through each tweet_id
for ind, tweet_id in enumerate(twitter_archive.tweet_id):
    public_metrics={}
    # store tweet_id
    public_metrics['tweet_id']=tweet_id
    # Basic request per tweet_id
    temp_data = client.get_tweet(id=tweet_id, tweet_fields=info)
    if temp_data.data:
        if temp_data.data.get('public_metrics'):
            # Loop through the public metrics dictionary to get the keys/values that are embedded
            for key, value in temp_data.data['public_metrics'].items():
                public_metrics[key]=value
    tweet_info.append(public_metrics)

Unauthorized: 401 Unauthorized

In [17]:
df1 = pd.read_csv('twitter-archive-enhanced.csv')
# extract values from tweet_id variable
tweet_ids = df1.tweet_id.values
# experiment with one tweet id
tweet_id=tweet_ids[0]
tweet_fields=["public_metrics"]
temp = client.get_tweet(id=tweet_id,  tweet_fields=tweet_fields)
# extract query data
tweet_data = temp.data
# store data
for key in tweet_fields:
    if tweet_data.get(key):
        print(tweet_data[key])

Unauthorized: 401 Unauthorized

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [13]:
twitter_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
111,871032628920680449,NaN,NaN,2017-06-03 15:55:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Boomer. He's doing an advanced water t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/871032628...,13,10,Boomer,None,None,None,None
1603,685943807276412928,NaN,NaN,2016-01-09 21:58:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is the newly formed pupper a capella grou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/685943807...,8,10,the,None,None,pupper,None
175,857989990357356544,NaN,NaN,2017-04-28 16:08:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rosie. She was just informed of the wa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/857989990...,12,10,Rosie,None,None,None,None
1776,677961670166224897,NaN,NaN,2015-12-18 21:20:32 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Izzy. She's showing off the dance move...,NaN,NaN,NaN,https://vine.co/v/iKuMDuYV0aZ,11,10,Izzy,None,None,None,None
1583,687102708889812993,NaN,NaN,2016-01-13 02:43:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Army of water dogs here. None of them know whe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/687102708...,5,10,None,None,None,None,None


In [42]:
twitter_predict.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1477,780543529827336192,https://pbs.twimg.com/media/CtUMLzRXgAAbZK5.jpg,1,golden_retriever,0.628312,True,Labrador_retriever,0.317365,True,Tibetan_mastiff,0.012260,True
1913,854010172552949760,https://pbs.twimg.com/media/C9oNt91WAAAFSLS.jpg,1,English_springer,0.354733,True,collie,0.177538,True,Border_collie,0.131706,True
96,667724302356258817,https://pbs.twimg.com/media/CUQ7tv3W4AA3KlI.jpg,1,ibex,0.619098,False,bighorn,0.125119,False,ram,0.074673,False
1468,778990705243029504,https://pbs.twimg.com/media/Cs-H5uhWcAAiNY9.jpg,2,cocker_spaniel,0.715351,True,Labrador_retriever,0.207056,True,Chihuahua,0.028519,True
903,700062718104104960,https://pbs.twimg.com/media/CbcfUxoUAAAlHGK.jpg,1,hummingbird,0.180998,False,peacock,0.135179,False,eel,0.075371,False


In [43]:
twitter_tweets.sample(5)

NameError: name 'twitter_tweets' is not defined

In [44]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [45]:
twitter_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [46]:
twitter_tweets.info()

NameError: name 'twitter_tweets' is not defined

In [47]:
list(twitter_predict)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

In [48]:
list(twitter_archive)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [49]:
list(twitter_tweets)

NameError: name 'twitter_tweets' is not defined

In [50]:
all_columns = pd.Series(list(twitter_archive) + list(twitter_predict) + list(twitter_tweets))
all_columns[all_columns.duplicated()]

NameError: name 'twitter_tweets' is not defined

In [51]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [52]:
twitter_predict.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [53]:
twitter_tweets.describe()

NameError: name 'twitter_tweets' is not defined

In [54]:
twitter_archive.name.value_counts()

None         745
a             55
Charlie       12
Cooper        11
Lucy          11
Oliver        11
Lola          10
Penny         10
Tucker        10
Winston        9
Bo             9
the            8
Sadie          8
an             7
Bailey         7
Daisy          7
Toby           7
Buddy          7
Bella          6
Scout          6
Milo           6
Oscar          6
Stanley        6
Dave           6
Jack           6
Koda           6
Rusty          6
Jax            6
Leo            6
Sunny          5
            ... 
Saydee         1
Daniel         1
Mya            1
Bloo           1
Suki           1
Pubert         1
Damon          1
Rolf           1
Heinrich       1
Emma           1
Ralf           1
Cermet         1
Toffee         1
Tanner         1
Ole            1
Meatball       1
Horace         1
Genevieve      1
Olaf           1
Murphy         1
Hermione       1
Snickers       1
Mollie         1
Stephanus      1
Iggy           1
Dallas         1
Kayla          1
Gabby         

In [55]:
twitter_archive['is_duplicated'].sum()

KeyError: 'is_duplicated'

In [56]:
twitter_archive[tweet_id.duplicated()]

AttributeError: 'int' object has no attribute 'duplicated'

In [57]:
twitter_predict.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
malamute                      30
cocker_spaniel                30
French_bulldog                26
Chesapeake_Bay_retriever      23
miniature_pinscher            23
seat_belt                     22
German_shepherd               20
Staffordshire_bullterrier     20
Siberian_husky                20
web_site                      19
Cardigan                      19
Shetland_sheepdog             18
teddy                         18
Eskimo_dog                    18
Maltese_dog                   18
beagle                        18
Rottweiler                    17
Lakeland_terrier              17
Shih-Tzu                      17
Italian_greyhound             16
kuvasz                        16
          

In [58]:
twitter_predict[twitter_predict.tweet_id.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [59]:
list(twitter_archive.text)[:5]

["This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 "This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",
 'This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB',
 'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ',
 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f']

In [60]:
twitter_archive.timestamp

0       2017-08-01 16:23:56 +0000
1       2017-08-01 00:17:27 +0000
2       2017-07-31 00:18:03 +0000
3       2017-07-30 15:58:51 +0000
4       2017-07-29 16:00:24 +0000
5       2017-07-29 00:08:17 +0000
6       2017-07-28 16:27:12 +0000
7       2017-07-28 00:22:40 +0000
8       2017-07-27 16:25:51 +0000
9       2017-07-26 15:59:51 +0000
10      2017-07-26 00:31:25 +0000
11      2017-07-25 16:11:53 +0000
12      2017-07-25 01:55:32 +0000
13      2017-07-25 00:10:02 +0000
14      2017-07-24 17:02:04 +0000
15      2017-07-24 00:19:32 +0000
16      2017-07-23 00:22:39 +0000
17      2017-07-22 16:56:37 +0000
18      2017-07-22 00:23:06 +0000
19      2017-07-21 01:02:36 +0000
20      2017-07-20 16:49:33 +0000
21      2017-07-19 16:06:48 +0000
22      2017-07-19 03:39:09 +0000
23      2017-07-19 00:47:34 +0000
24      2017-07-18 16:08:03 +0000
25      2017-07-18 00:07:08 +0000
26      2017-07-17 16:17:36 +0000
27      2017-07-16 23:58:41 +0000
28      2017-07-16 20:14:00 +0000
29      2017-0

In [61]:
twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [62]:
twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

### Quality issues
1. twitter_archive has some duplicate tweets

2. twitter_archive text column contains hyperlink which we do not need 

3. twitter_archive timestamp is data type object and tweet_id is integer

4. twitter_predict shows several predictions but only one would be needed

5. twitter_archive has unnecessary columns like in_reply_to_status_idm in_reply_to_user_id, retweeted_status_idm source, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp

6. twitter_archive - some rows in column name contain 'None' - missing data

7. twitter_archive - some rows in column name contains non-names values with lower case - like a or quite

8. twitter_tweets columns, despite being numbers, are ‘object’ Dtype and the shared observation is labeled tweet ID 

9. twitter_archive rating_numerator has values ranging from 0 to 1779 and denominator 0-170

### Tidiness issues
1. twitter_tweets and twitter_predict have certain some observations as twitter_archive - they do not need to be separate observations

2. twitter_archive has column headers doggo, flooder, pepper, puppy - these could be better as one variable in one column not as separate columns

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [22]:
# Make copies of original pieces of data
twitter_archive_clean = twitter_archive.copy()
twitter_predict_clean = twitter_predict.copy()
#twitter_tweets_clean = twitter_tweets.copy()

### Issue #1:

#### Define: twitter_archive has some duplicate tweets
Remove unnecessary retweets using boolean masking to select only entries that have null values (ie. that are "True") for retweeted_status_id

#### Code

In [23]:
# Total number of tweets including retweets
twitter_archive_clean.shape[0]

2356

In [24]:
# Number of retweets
twitter_archive_clean[twitter_archive_clean.retweeted_status_id.isnull()== False].count()[0]

181

In [25]:
# remove duplicates
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.retweeted_status_id.isnull()== True]

#### Test

In [26]:
# Total number of tweets (should be  2175 [2356 - 181])
twitter_archive_clean.shape[0]

2175

### Issue #2:

#### Define - twitter_archive text column contains hyperlink which we do not need 
Remove hyperlink data from text column in the twitter_archive dataframe

#### Code

In [27]:
# Check if 'text' columns contain URLs (should be "True")
string = twitter_archive_clean.text.str.contains(r'\shttps.+$').describe()[2]
print("Does the `text` column contain URLs that should be removed?: {}".format(string))

Does the `text` column contain URLs that should be removed?: True


In [28]:
# Precleaning check
list(twitter_archive_clean.text)[:5]

["This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 "This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",
 'This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB',
 'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ',
 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f']

In [29]:
# Remove URLs
twitter_archive_clean["text"] = twitter_archive_clean.text.str.split(r'\shttps.+$', expand=True)

#### Test

In [30]:
# Check if 'text' columns contain URLs (should be "True")
string = twitter_archive_clean.text.str.contains(r'\shttps.+$').describe()[2]
print("Does the `text` column contain URLs that should be removed?: {}".format(string))

Does the `text` column contain URLs that should be removed?: False


In [31]:
# Precleaning check
list(twitter_archive_clean.text)[:5]

["This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10",
 "This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10",
 'This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10',
 'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us',
 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek']

In [32]:
# Remove URLs
twitter_archive_clean["text"] = twitter_archive_clean.text.str.split(r'\shttps.+$', expand=True)

#### Test

In [33]:
# Check if 'text' columns contain URLs (should be "False")
string = twitter_archive_clean.text.str.contains(r'\shttps.+$').describe()[2]
print("Does the `text` column contain URLs that should be removed?: {}".format(string))

Does the `text` column contain URLs that should be removed?: False


In [34]:
# Postcleaning check
list(twitter_archive_clean.text)[:5]

["This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10",
 "This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10",
 'This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10',
 'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us',
 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek']

### Issue #3:

#### Define - twitter_archive timestamp is data type object and tweet_id is integer
Change dtype of timestamp column to datatime using to_datetime

#### Code

In [60]:
# Precleaning check
twitter_archive_clean["timestamp"].dtypes

dtype('<M8[ns]')

In [59]:
twitter_archive_clean["tweet_id"].dtypes

dtype('int64')

In [37]:
# Change dtype of `timestamp` column to `datatime` using `to_datetime`
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])

In [61]:
# Change tweet_id from integer to string
twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype('str')

#### Test

In [38]:
# check code worked
twitter_archive_clean["timestamp"].dtypes

dtype('<M8[ns]')

In [62]:
# check code worked
twitter_archive_clean["tweet_id"].dtypes

dtype('O')

### Issue #4:

#### Define twitter_predict shows several predictions but only one would be needed
I reviewed the data and decided to only keep the first prediction as that is usually the best one

#### Code

In [39]:
#  Precleaning check
list(twitter_predict_clean.columns)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

In [40]:
twitter_predict_clean.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [41]:
# Subset out unnecessary columns
twitter_predict_clean = twitter_predict_clean[['tweet_id', 'jpg_url', 'p1']]

In [42]:
# Rename column
twitter_predict_clean = twitter_predict_clean.rename(columns = {"p1":"breed"})

#### Test

In [83]:
# Postcleaning check
list(twitter_predict_clean.columns)

['tweet_id', 'jpg_url', 'breed']

In [43]:
twitter_predict_clean.head(3)

,tweet_id,jpg_url,breed
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,Welsh_springer_spaniel
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,redbone
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,German_shepherd


### Issue #5:

#### Define twitter_archive has unnecessary columns like in_reply_to_status_idm in_reply_to_user_id, retweeted_status_idm source, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp
Drop the not needed columns

#### Code

In [44]:
list(twitter_archive_clean)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [45]:
# Drop the not needed columns
unnecessary_columns = ["in_reply_to_status_id", 
                       "in_reply_to_user_id", 
                       "retweeted_status_id", 
                       "source", 
                       "retweeted_status_id", 
                       "retweeted_status_user_id",
                       "retweeted_status_timestamp"]

twitter_archive_clean = twitter_archive_clean.drop(unnecessary_columns, axis=1) 

#### Test

In [46]:
list(twitter_archive_clean)

['tweet_id',
 'timestamp',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

### Issue #6:

#### Define twitter_archive - some rows in column name contain 'None' - missing data
fix the names in the column name to contain only valid data

#### Code

In [47]:
# this one is now easier to do on the final cleaned data than if we tried it in the beginning
twitter_archive_clean.name.value_counts().nlargest(50)

None        680
a            55
Charlie      11
Lucy         11
Cooper       10
Oliver       10
Penny         9
Tucker        9
Winston       8
Sadie         8
the           8
Lola          8
Daisy         7
Toby          7
Stanley       6
Oscar         6
Koda          6
Jax           6
an            6
Bailey        6
Bo            6
Bella         6
Milo          5
Dave          5
Leo           5
Buddy         5
Louis         5
Rusty         5
Scout         5
Chester       5
Bentley       5
Boomer        4
Chip          4
Gus           4
Bear          4
Sophie        4
Finn          4
very          4
Clarence      4
Dexter        4
Winnie        4
Scooter       4
Maggie        4
Phil          4
Jeffrey       4
Cassie        4
Reggie        4
Sammy         4
Clark         4
Jack          4
Name: name, dtype: int64

In [48]:
# Get count of misentered entries - starting small letters
string = twitter_archive_clean.name[twitter_archive_clean.name.str.match(r'(^[a-z])')].count()
print("There are {} invalid names in the `name` column".format(string))

There are 104 invalid names in the `name` column


In [66]:
# Isolate entries of "None" in the `name` column and subset the `name` and `text` columns for easier viewing
none_names = twitter_archive_clean.query('name == "None"')[["name","text"]]
none_names.head()

,name,text


In [ ]:
string4 = none_names.count()[0]
print("There are {} entries with 'None' entered as the dog's name.".format(string4))

In [49]:
# first, let's change "None" to real None
twitter_archive_clean.name.replace('None', inplace=True)

In [50]:
# it looks like all names starting with lowercase letter seem not to be real names - let's check more in detail
empty=twitter_archive_clean.loc[twitter_archive_clean['name'].str.islower()][['tweet_id','text','name']]

In [51]:
from IPython.display import display
display(empty.style.set_properties(**{
        'width': '1000px',
        'max-width': '1000px'
    }))

,tweet_id,text,name
22,887517139158093824,I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy),such
56,881536004380872706,Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af (IG: puffie_the_chow),a
169,859196978902773760,We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10,quite
170,859074603037188101,Here we have an exotic dog. Good at ukulele. Fashionable af. Has two more arms if needed. Is blue. Knows what 'ohana means. 13/10 would pet,quite
172,858843525470990336,I have stumbled puppon a doggo painting party. They're looking to be the next Pupcasso or Puppollock. All 13/10 would put it on the fridge,quite
193,855459453768019968,"Guys, we only rate dogs. This is quite clearly a bulbasaur. Please only send dogs. Thank you... 12/10 human used pet, it's super effective",quite
335,832645525019123713,There's going to be a dog terminal at JFK Airport. This is not a drill. 10/10,not
369,828650029636317184,"Occasionally, we're sent fantastic stories. This is one of them. 14/10 for Grace",one
542,806219024703037440,We only rate dogs. Please stop sending in non-canines like this Freudian Poof Lion. This is incredibly frustrating... 11/10,incredibly
649,792913359805018113,Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest,a


**Notes:**
- most of these texts do not actually contain any names - so should be changed to None

- some have name but not extracted correctly 

- some were non-dog tweets

In [ ]:
# First Removing tweets that do not rate dogs and non-dog tweets or unclear tweets
twitter_archive_clean.drop([169,193,335,369,542,801,819,988,1002,
                            1004,1017,1019,1018,1025,1031,1071,1097,
                            1098, 1121,1193,1206,1207,1259,1260, 1341,
                           1352,1353,1361,1362,1385,1435,1527,1693,1737,
                           1854,1877,1942,1943,], axis=0, inplace=True)

In [67]:
# now use a function to look for names with None and replace with actual name, the actual name will be placed in column dog_name
def dog_name_finder(df):
    """ Use a regex to extract the name of the dog and place the dog name in the 'name' column.
    
    Keyword arguments:
    df -- String in 'text' column must contain the one of the following patterns to be matched:
    - "named *dog name*"
    - "name is *dog name*" 
    
    """
    x = df.text.str.extract(r'(?:(?:name\sis\s)|(?:named\s))([a-zA-Z]+)')
    df['dog_name'] = x[0]
    df = df[x[0].isnull() == False]
    return df

none_names = dog_name_finder(none_names)
none_names

,name,text,dog_name


#### Test

### Issue #7:

#### Define

#### Code

#### Test

### Issue #8:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization